In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
!cd yolov5

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [ ]:
!pip install -r /content/yolov5/requirements.txt

In [ ]:
!python /content/yolov5/detect.py --weights best.pt --img 640 --conf 0.25 --source /content/KakaoTalk_20250703_175530256.mp4 --save-txt

detect: weights=['best.pt'], source=/content/KakaoTalk_20250703_175530256.mp4, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-422-g2540fd4c Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/362) /content/KakaoTalk_20250703_175530256.mp4: 480x640 1 out of water, 29.2ms
video 1/1 (2/362) /content/KakaoTalk_20250703_175530256.mp4: 480x640 1 out of water, 9.1ms
video 1/1 (3/362) /content/KakaoTalk_20250703_175530256.mp4: 480x640 1 out of water, 9.0ms
video 1/1 (4/362) /content/K

In [ ]:
import cv2
import os
from collections import deque
import numpy as np
import time

# 경로 설정
video_path = '/content/KakaoTalk_20250703_175530256.mp4'  # 원본 영상 경로
output_video_path = '/content/output_video.mp4'  # 출력 영상 경로
labels_dir = '/content/yolov5/runs/detect/exp/labels/'  # YOLOv5 탐지 결과 폴더

# 영상 속성 가져오기
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# 3초에 해당하는 프레임 수 계산
N = int(fps * 3)

# 클래스 0 탐지 여부를 저장할 deque 초기화
class0_history = deque(maxlen=N)

# 출력 영상 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# 커스텀 디자인 함수 정의
def draw_custom_design(frame, bbox, class_id, warning=False):
    x_min, y_min, x_max, y_max = bbox
    if warning:
        current_time = time.time()
        # 워닝 스타일: 빨간색 사각형과 "Warning" 텍스트
        blink_intensity = int(255 * abs(np.sin(current_time * 40)))
        blink_color = (0, 0, blink_intensity)
        cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), blink_color, 2)
        warning_text = "WARNING"
        text_size = cv2.getTextSize(warning_text, cv2.FONT_HERSHEY_SIMPLEX, 1.5, 3)[0]
        text_x = x_min + (x_max - x_min - text_size[0]) // 2
        text_y = y_min + (y_max - y_min + text_size[1]) // 2
        cv2.putText(frame, warning_text, (text_x, text_y),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.5, blink_color, 3, cv2.LINE_AA)

    elif class_id == 2:
        # 기본 스타일: 초록색 사각형과 클래스 이름
        cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (255, 255, 255), 2)
        label = 'Out of Water'
        cv2.putText(frame, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    elif class_id == 0:
        # 기본 스타일: 초록색 사각형과 클래스 이름
        cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 0, 255), 2)
        label = 'Drowning'
        cv2.putText(frame, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    elif class_id == 1:
        # 기본 스타일: 초록색 사각형과 클래스 이름
        cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (255, 150, 70), 2)
        label = 'Swimming'
        cv2.putText(frame, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 150, 70), 2)

# 프레임 처리
frame_id = 1
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 워닝 모드 결정 (과거 프레임 기준)
    warning_mode = len(class0_history) > 0 and (sum(class0_history) / len(class0_history) > 0.7)

    # 현재 프레임의 클래스 0 탐지 여부 초기화
    has_class0 = False

    # 해당 프레임의 탐지 결과 파일 읽기
    label_file = os.path.join(labels_dir, f'KakaoTalk_20250703_175530256_{frame_id}.txt')
    if os.path.exists(label_file):
        with open(label_file, 'r') as f:
            lines = f.readlines()
            for line in lines:
                parts = line.strip().split()
                class_id = int(parts[0])
                if class_id == 0:
                    has_class0 = True
                # YOLO 형식 좌표를 픽셀 좌표로 변환
                x_center = float(parts[1]) * frame_width
                y_center = float(parts[2]) * frame_height
                width = float(parts[3]) * frame_width
                height = float(parts[4]) * frame_height
                x_min = int(x_center - width / 2)
                y_min = int(y_center - height / 2)
                x_max = int(x_center + width / 2)
                y_max = int(y_center + height / 2)
                bbox = [x_min, y_min, x_max, y_max]
                # 커스텀 디자인 그리기
                draw_custom_design(frame, bbox, class_id, warning=(class_id == 0 and warning_mode))

    # 현재 프레임의 탐지 여부를 히스토리에 추가
    class0_history.append(has_class0)

    # 처리된 프레임을 출력 영상에 추가
    out.write(frame)
    frame_id += 1

# 리소스 해제
cap.release()
out.release()